# Digital PDF Signing with p12 Certificate
First, let's install required packages

In [1]:
!pip install pypdf reportlab endesive oscrypto
!apt-get update && apt-get install -y libssl-dev openssl swig python3-dev


E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)


Import necessary libraries and set up timestamp

In [2]:
from pypdf import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import io
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
signature_text = f"Digitally signed by R. Haase, {timestamp}"

Create visible signature watermark

In [3]:
packet = io.BytesIO()
c = canvas.Canvas(packet, pagesize=letter)
c.setFont("Helvetica", 12)
c.setFillColorRGB(1, 0, 0)
c.drawString(50, 50, signature_text)
c.save()
packet.seek(0)

0

Add visible signature to PDF

In [4]:
existing_pdf = PdfReader("test2.pdf")
signature_pdf = PdfReader(packet)
output = PdfWriter()

page = existing_pdf.pages[0]
page.merge_page(signature_pdf.pages[0])
output.add_page(page)

for page in existing_pdf.pages[1:]:
    output.add_page(page)

output.write("temp_signed.pdf")

(True, <_io.FileIO [closed]>)

Add cryptographic signature using p12 certificate

In [5]:
from endesive import pdf

date = datetime.utcnow()
date_str = date.strftime("%Y%m%d%H%M%S+00'00'")

with open("certificate.p12", "rb") as fp:
    p12 = fp.read()

dct = {
    "sigflags": 3,
    "contact": "R. Haase",
    "location": "Dresden",
    "signingdate": date_str,
    "reason": "Digital Signature",
}

with open("temp_signed.pdf", "rb") as fp:
    datau = fp.read()

signature = pdf.cms.sign(datau, dct,
    p12,
    "certificate_password",  # Replace with actual password
    "sha256"
)

with open("signed_test2.pdf", "wb") as fp:
    fp.write(datau)
    fp.write(signature)

LibraryNotFoundError: Error detecting the version of libcrypto